In [1]:
import pandas as pd
import torch
from chronos import ChronosPipeline
import numpy as np
import timesfm

from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame


pipeline_choronos_t5_large = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-large",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="cuda:0",
          per_core_batch_size=32,
          horizon_len=50,
          input_patch_len=32,
          output_patch_len=128,
          num_layers=50,
          model_dims=1280,
          use_positional_embedding=False,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )


def split_series(series, segment_len=500):
    if isinstance(series, list):
        series = torch.tensor(series, dtype=torch.float32)
    elif isinstance(series, torch.Tensor):
        series = series.to(torch.float32)
    else:
        raise ValueError("Input must be list or tensor")
    
    n = len(series)
    segments = []
    for start in range(0, n - segment_len + 1, segment_len):
        segments.append(series[start:start + segment_len])
    return segments

def predict_next_50(time_series, pipeline):
    if isinstance(time_series, list):
        time_series = torch.tensor(time_series, dtype=torch.float32)
    elif isinstance(time_series, torch.Tensor):
        time_series = time_series.to(torch.float32)
    else:
        raise ValueError("Input must be list or tensor")
    
    if time_series.ndim != 1:
        raise ValueError("Input series must be 1D")
    if len(time_series) < 450:
        raise ValueError("Input series length must be at least 450")
    
    input_segment = time_series[:450].unsqueeze(0)
    forecast = pipeline.predict(input_segment, prediction_length=50)
    return forecast.squeeze(0).squeeze(0).detach().cpu().numpy()

def predict_next_50_bolt(time_series, pipeline):
    if isinstance(time_series, list):
        time_series = torch.tensor(time_series, dtype=torch.float32)
    elif isinstance(time_series, torch.Tensor):
        time_series = time_series.to(torch.float32)
    else:
        raise ValueError("Input must be list or tensor")
    
    if time_series.ndim != 1:
        raise ValueError("Input series must be 1D")
    if len(time_series) < 450:
        raise ValueError("Input series length must be at least 450")
    
    # Formalized it
    input_segment = pd.DataFrame(time_series[:450].numpy(), columns=["target"])
    input_segment["item_id"] = "series_basic"
    input_segment["timestamp"] = pd.date_range(start="2020-01-01", periods=len(input_segment), freq="D")
    input_segment = input_segment[["item_id", "timestamp", "target"]]
    
    predictor  = pipeline.fit(
        input_segment,
        hyperparameters={
            "Chronos": {"model_path": "amazon/chronos-bolt-base"},
        },
    )
    predictions = predictor.predict(input_segment)
    return predictions["mean"].to_numpy()

from sklearn.linear_model import LinearRegression
def linear_trend_forecast(series, horizon):
    series = np.asarray(series)
    n = len(series)
    X = np.arange(n).reshape(-1,1)
    y = series
    model = LinearRegression()
    model.fit(X, y)
    X_future = np.arange(n, n+horizon).reshape(-1,1)
    forecast = model.predict(X_future)
    return forecast


def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # 避免除零，加个小常数
    return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), 1e-8, None))) * 100


def sMAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2

    denominator = np.clip(denominator, 1e-8, None)
    return np.mean(np.abs(y_true - y_pred) / denominator) * 100


def weighted_quantile_loss(y_true, y_pred, quantile=0.5, weight=1.0):

    diff = y_true - y_pred
    return np.mean(weight * np.maximum(quantile * diff, (quantile - 1) * diff))

def Agg_Relative_WQL(y_true, y_pred, base_pred, quantiles=[0.1, 0.5, 0.9], weights=None):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    base_pred = np.array(base_pred)
    if weights is None:
        weights = np.ones(len(quantiles)) / len(quantiles)
    agg_relative_wql_sum = 0
    for q, w in zip(quantiles, weights):
        q_pred = y_pred
        base_q_pred = base_pred
        wql = weighted_quantile_loss(y_true, q_pred, quantile=q)
        base_wql = weighted_quantile_loss(y_true, base_q_pred, quantile=q)
        agg_relative_wql_sum += w * (wql / (base_wql + 1e-8))
    return agg_relative_wql_sum


def MASE(y_true, y_pred, y_train, seasonality=1):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_train = np.array(y_train)
    n = len(y_train)
    d = np.abs(y_train[seasonality:] - y_train[:-seasonality]).mean()
    errors = np.abs(y_true - y_pred)
    return errors.mean() / (d + 1e-8)

def Agg_Relative_MASE(y_true, y_pred, base_pred, y_train, seasonality=1):
    model_mase = MASE(y_true, y_pred, y_train, seasonality)
    base_mase = MASE(y_true, base_pred, y_train, seasonality)
    return model_mase / (base_mase + 1e-8)


def predict_long_series(series):

    seasonality = 1
    segments = split_series(series, 500)
    
    all_metrics = []
    seg_id = 0
    for seg in segments:
        # This is for amazon/chronos-bolt-base
        pipeline_choronos_bolt_base = TimeSeriesPredictor(prediction_length=50)
        pred_choronos_bolt = predict_next_50_bolt(seg[:450], pipeline_choronos_bolt_base)

        # This is for amazon/chronos-t5-large
        pred_choronos_t5 = predict_next_50(seg[:450], pipeline_choronos_t5_large)

        # This is for linear tnd model
        pred_linear_trend = linear_trend_forecast(seg[:450].cpu().numpy(), 50)

        # This is for timesfm
        input_tensor = seg[:450].unsqueeze(0).detach().cpu().numpy()
        pred_timesfm = np.squeeze(tfm.forecast(input_tensor, [0])[0])
    
        y_train = seg[:450].cpu().numpy()
        ground_truth = seg[450:].cpu().numpy()
        
        segment_metrics = {}
        for name, pred in [
                ('chronos_bolt', pred_choronos_bolt),
                ('chronos_t5', pred_choronos_t5),
                ('linear_trend', pred_linear_trend),
                ('timesfm', pred_timesfm)
        ]:
            mape = MAPE(ground_truth, pred)
            smape = sMAPE(ground_truth, pred)
            agg_wql = Agg_Relative_WQL(ground_truth, pred, pred_linear_trend) 
            agg_mase = Agg_Relative_MASE(ground_truth, pred, pred_linear_trend, y_train, seasonality)

            print(f"{name} - MAPE: {mape:.4f}%, sMAPE: {smape:.4f}%, Agg. Relative WQL: {agg_wql:.4f}, Agg. Relative MASE: {agg_mase:.4f}\n")
            segment_metrics[name] = {
                "MAPE": mape,
                "sMAPE": smape,
                "Agg_Relative_WQL": agg_wql,
                "Agg_Relative_MASE": agg_mase,
            }
            segment_metrics["seg_id"] = seg_id
        all_metrics.append(segment_metrics)
        seg_id += 1
    return all_metrics

/home/snt/projects_lujun/benchmarking_nature_tsfm/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.10.18 (main, Jul 11 2025, 22:43:29) [Clang 20.1.4 ].


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 42281.29it/s]


In [2]:
import pandas as pd 
from tqdm import tqdm

df = pd.read_csv("/home/snt/projects_lujun/benchmarking_nature_tsfm/data/M/M4-methods/Dataset/Train/Daily-train.csv")

output_path = "/home/snt/projects_lujun/benchmarking_nature_tsfm/data/dataset/Daily-train_predicted.jsonl"

In [3]:
def extract_time_series(row):
    return row.loc['V2':].dropna().tolist()

df_optimize = pd.DataFrame()
df_optimize['unique_id'] = df['V1']
df_optimize['target'] = df.apply(extract_time_series, axis=1)
df = df_optimize.copy()

df = df[:540]


In [ ]:
def flatten_dict(d):
    flat = {}
    for key, val in d.items():
        if isinstance(val, dict):
            for sub_key, sub_val in val.items():
                flat[f'{key}_{sub_key}'] = sub_val
        else:
            flat[key] = val
    return flat

all_results = pd.DataFrame()

for index, row in tqdm(df.iterrows(), total=len(df)):
    if index == 0:
        mode = "w"
    else:
        mode = "a"
    series = row["target"]
    all_metrics = predict_long_series(series)
    flat_data = [flatten_dict(d) for d in all_metrics]
    results_df = pd.DataFrame(flat_data)
    results_df["unique_id"] = row["unique_id"]
    results_df.to_json(output_path, lines=True, mode=mode, orient='records')


In [6]:
import pandas as pd

df = pd.read_json(output_path, lines=True)

In [7]:
df

,chronos_bolt_MAPE,chronos_bolt_sMAPE,chronos_bolt_Agg_Relative_WQL,chronos_bolt_Agg_Relative_MASE,seg_id,chronos_t5_MAPE,chronos_t5_sMAPE,chronos_t5_Agg_Relative_WQL,chronos_t5_Agg_Relative_MASE,linear_trend_MAPE,linear_trend_sMAPE,linear_trend_Agg_Relative_WQL,linear_trend_Agg_Relative_MASE,timesfm_MAPE,timesfm_sMAPE,timesfm_Agg_Relative_WQL,timesfm_Agg_Relative_MASE,unique_id
0,0.626232,0.625091,0.519132,0.187506,0,2.183018,2.192619,1.177317,0.657847,3.331036,3.389682,1.0,1.0,1.114767,1.106562,1.108498,0.332130,D1
1,1.658555,1.641046,1.807619,1.974376,1,2.270605,2.276095,4.471470,2.718531,0.842893,0.837511,1.0,1.0,0.613837,0.611329,0.812345,0.736417,D1
2,1.419748,1.405738,1.538162,1.745475,0,2.265495,2.226628,2.569713,2.795078,0.811415,0.808607,1.0,1.0,0.784555,0.780854,0.903167,0.965858,D2
3,3.115892,3.045503,0.196366,0.189454,1,6.370142,6.304914,0.672526,0.392887,16.249364,15.006401,1.0,1.0,0.850985,0.851673,0.126197,0.052528,D2
4,7.282652,7.717092,0.844055,0.844055,0,10.403984,11.292234,1.208568,1.189509,8.709090,9.263561,1.0,1.0,3.599397,3.703369,0.502323,0.416694,D6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1082,2.414961,2.376834,0.805264,0.801545,3,4.441322,4.391322,2.487801,1.483853,3.006481,2.952943,1.0,1.0,0.915468,0.920153,0.869971,0.307678,D539
1083,3.695296,3.796643,0.948163,0.935272,0,4.561486,4.725113,1.343145,1.141723,3.933748,4.064988,1.0,1.0,0.791792,0.789132,0.342400,0.194448,D540
1084,1.076147,1.067548,0.144421,0.089400,1,5.444974,5.696187,1.372022,0.456829,12.014116,11.321310,1.0,1.0,1.357480,1.342116,0.124005,0.112225,D540
1085,3.665116,3.747455,0.961670,0.285331,2,3.828201,3.910054,0.846068,0.297007,12.994446,12.187137,1.0,1.0,1.281978,1.292996,0.318284,0.099781,D540


In [10]:
import pandas as pd

# 假设你的DataFrame是 df
# 计算除 'unique_id' 和 'seg_id' 列外，其余列的平均值
mean_values = df.drop(columns=['unique_id', 'seg_id']).mean()

# 如果需要将结果转换成DataFrame格式并输出
mean_df = mean_values.to_frame(name='mean').reset_index()
print(mean_df)

                             index       mean
0                chronos_bolt_MAPE   4.926186
1               chronos_bolt_sMAPE   5.026335
2    chronos_bolt_Agg_Relative_WQL   1.002388
3   chronos_bolt_Agg_Relative_MASE   0.633483
4                  chronos_t5_MAPE   7.108406
5                 chronos_t5_sMAPE   7.175055
6      chronos_t5_Agg_Relative_WQL   1.559333
7     chronos_t5_Agg_Relative_MASE   0.978574
8                linear_trend_MAPE  14.139789
9               linear_trend_sMAPE  14.602849
10   linear_trend_Agg_Relative_WQL   1.000000
11  linear_trend_Agg_Relative_MASE   1.000000
12                    timesfm_MAPE   1.899082
13                   timesfm_sMAPE   2.034422
14        timesfm_Agg_Relative_WQL   0.389583
15       timesfm_Agg_Relative_MASE   0.228565
